In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('unsloth/Llama-3.2-3B-Instruct')

In [4]:
streamer = TextStreamer(tokenizer)

In [5]:
ori_model = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-3B-Instruct', tie_word_embeddings=False
).cuda()

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at unsloth/Llama-3.2-3B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
ori_model.lm_head.weight.data = ori_model.model.embed_tokens.weight.data.clone()

In [7]:
state_dict = ori_model.state_dict()

In [8]:
from huggingface_hub import snapshot_download

lora = snapshot_download(
    repo_id="mesolitica/lora-embedding-256-llama3.2-3b-multipack"
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
from safetensors import safe_open

f = safe_open(f"{lora}/adapter_model.safetensors", framework="pt", device=0)

In [10]:
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [11]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A)
    B = f.get_tensor(B)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 478.95it/s]


In [12]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#     'mesolitica/lora-embedding-256-llama3.2-3b-multipack', torch_dtype=torch.bfloat16,
# )
# _ = model.cuda()

In [13]:
d = [
    {'role': 'user', 'content': 'ape isu dengan najib'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.0,
    do_sample=False,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you m

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

ape isu dengan 

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


najib<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Najib Razak, yang berkhidmat sebagai Perdana Menteri Malaysia dari 2009 hingga 2018, telah menjadi subjek pelbagai kontroversi dan skandal semasa memegang jawatan. Berikut adalah beberapa isu utama yang berkaitan dengannya:

1. Skandal 1MDB: Najib Razak terlibat dalam skandal kewangan besar-besaran yang melibatkan dana pelaburan negara 1Malaysia Development Berhad (1MDB). Jabatan Kehakiman AS mendakwa bahawa lebih $4.5 bilion telah diselewengkan daripada 1MDB, dengan beberapa dana mengalir ke akaun bank peribadi Najib. Najib secara konsisten menafikan sebarang salah laku, tetapi dia didakwa dengan pelbagai pertuduhan rasuah, pengubahan wang haram, dan penyalahgunaan kuasa di Malaysia.

2. Penindasan kebebasan bersuara dan perhimpunan: Semasa memegang jawatan, Najib Razak dikritik kerana pendekatannya yang keras terhadap kebebasan bersuara dan perhimpunan. Kerajaannya dituduh menggunakan pelbagai taktik untuk menyekat perbeza

In [14]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in jawi'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

/home/husein/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in jawi<|eot_id|><|start_header_id|>assistant<|end_header_id|>

اونتوق مڠکودکن رياکت، اندا بوليه ممولاکن دڠن مماسڠ ڤاکيج يڠ دڤرلوکن. بريکوت اداله چارا اونتوق منچاري دان مماسڠ ڤاکيج يڠ دڤرلوکن:

١. Node.js (ۏرسي ٨ اتاو لبيه تيڠݢي)
٢. npm (Node Package Manager)

جالنکن ارهن برايکوت اونتوق مماسڠ ڤاکيج يڠ دڤرلوکن:

```
npm install react react-dom
```

سکارڠ، چيڤتا فايل بارو يڠ دڤڠݢيل `App.js` دان تمبهکن کود برايکوت:

```jsx
import React from'react';

function App() {
  return (
    <div className="App">
      <h1>Hello World!</h1>
    </div>
  );
}

export default App;
```

دالم کود اين، کامي مڠايمڤورت کبرݢنتوڠن `React` دري ڤاکيج `react`. کمودين، کامي منتوکن فوڠسي `App` يڠ مڠمباليکن ايليمن `<div>` يڠ مڠندوڠي تاجوق "Hello World!".

اخير سکالي، کامي مڠيکسڤورت فوڠسي `App` سباݢاي اليرن اوتام.



In [16]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in sabah'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.3,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in sabah<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bah, kunuk sia mau kasi tau macam mana mo bikin kod React dalam Bahasa Kebangsaan Sunda. Tingu saja la ni contoh kode yang boleh kamurang guna:

```jsx
import React from'react';
// Import anu-anu lain kalau perlu

function KomponenApaSia() {
    // Kod komponen di sini
}

const Pengawal = () => (
    <div>
        {/* Susun barang-barang di situlah */}
        
    }
);

export default Pengawal;
```

Dalam kes tersebut tu `KomponenApa`, ada fungsi untuk apa-apa benda yang mungkin mo kita laksana.

Macam `Pengawal` pun sama ja bah, tapi ia macam pengurus utama yang ambil alih semua masalah-masalah besar dengan urus keadaannya pakai ciri-ciri 'State' dari pustaka'middleware'.

Jadi tugas kamu dorangan nanti sekarang lah supa

In [17]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in mandarin'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk selesaikan masalah hutang negara, reply in mandarin<|eot_id|><|start_header_id|>assistant<|end_header_id|>

解决马来西亚的国家债务问题需要采取多方面的方法，包括财政改革、经济增长和投资。以下是一些可能的策略：

1. 改革财政制度：马来西亚可以通过减少政府支出、提高收入和实施更有效的公共部门管理来改善其财政状况。这可能涉及缩小政府的规模、减少不必要的公共项目以及促进私营部门的增长。
2. 增加税收：马来西亚可以通过增加税收并提高税收效率来增加收入。这可能涉及对富裕个人和公司征收更高的税率，以及实施新的税收机制，如销售税或消费税。
3. 投资长期计划：马来西亚可以通过投资基础设施、教育和医疗保健等长期计划来创造就业机会并刺激经济增长。这可能涉及向私营部门提供补贴以鼓励投资，也可能涉及为这些领域进行大量公共投资。
4. 实施财政监管改革：马来西亚可以通过实施财政监管改革来提高其财政绩效。这可能涉及建立独立的财政机构来监督政府财政，并实施透明度和问责制措施。
5. 与国际伙伴合作：马来西亚可以与国际组织和其他国家合作解决国家债务问题。这可能涉及寻求援助和支持，或参与国际金融机构的举措。

值得注意的是，这些只是马来西亚可以采取的一些潜在策略，解决国家债务问题是一个复杂而有挑战性的任务，需要采取全面和协调的方法。<|eot_id|>


In [18]:
d = [
    {'role': 'user', 'content': 'camne nk code react'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Untuk memulakan, anda perlu mempunyai persekitaran React yang dipasang pada komputer atau mesin anda. Berikut adalah langkah-langkah asas untuk menulis kod React:

1. **Pasang Node.js dan npm**: Pastikan anda telah memasang Node.js (versi 14 ke atas) dan npm (Node Package Manager) di komputer anda.

2. **Cipta projek baru dengan `create-react-app`**:
   - Buka terminal atau command prompt.
   - Jalankan arahan berikut untuk mencipta projek React baru menggunakan `create-react-app`:
     ```
     npx create-react-app nama-projek-anda
     ```
     Gantikan "nama-projek-anda" dengan nama sebenar untuk projek anda.

3. **Navigasi ke direktori projek**:
   - Selepas penciptaan, navigasi ke direktori projek dengan menjalankan:
     ```
     cd

In [19]:
ori_model = ori_model.to(torch.bfloat16)

In [21]:
ori_model.push_to_hub('mesolitica/malaysian-Llama-3.2-3B-Instruct')

README.md:   0%|          | 0.00/673 [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-Llama-3.2-3B-Instruct/commit/d85cca4fb40612d5f3a80baeea5ccaed193b93b3', commit_message='Upload LlamaForCausalLM', commit_description='', oid='d85cca4fb40612d5f3a80baeea5ccaed193b93b3', pr_url=None, pr_revision=None, pr_num=None)